In [5]:
import arcpy
import os
import time

def generate_service_area(mode, field_name, output_file_name, facilities):
    # Record the start time
    start_time = time.time()

    # Set the workspace and overwrite output
    workspace = r"E:\STP_PWoD\MySTPProject\MySTPProject.gdb"
    arcpy.env.workspace = workspace
    arcpy.env.overwriteOutput = True

    # Specify the network dataset
    network_dataset = r"E:\\STP_9_22_2023\\STP\\MM_NetworkDataset_03092022.gdb\\NetworkDataset\\NetworkDataset_ND"
    travel_mode = mode  # Set the travel mode based on the passed mode parameter

    # Output directory path
    output_dir = rf"E:\STP_PWoD\ServiceArea_Tbig_Tsm\Origin_Dest_Tbg_Tsm\SerArea_{mode}"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)

    # Create an empty feature class to store all service areas
    service_area_fc_name = f'ServiceArea_{mode}_{field_name}'
    service_area_fc = arcpy.CreateFeatureclass_management(workspace, service_area_fc_name, 'POLYGON').getOutput(0)

    # Define the coordinate system for the ServiceArea feature class
    sr = arcpy.SpatialReference("NAD 1983 UTM Zone 12N")
    arcpy.DefineProjection_management(service_area_fc, sr)

    # Add fields to store the 'hptripid' and field_name in the ServiceArea feature class
    arcpy.AddField_management(service_area_fc, "hptripid", "TEXT")
    arcpy.AddField_management(service_area_fc, "travel_time_budget", "DOUBLE")

    # Iterate through each person in the facilities feature class
    with arcpy.da.SearchCursor(facilities, ['hptripid', field_name]) as cursor:
        for row in cursor:
            id, travel_time_budget = row

            # Check if travel_time_budget is greater than zero
            if travel_time_budget <= 0:
                print(f"Skipping facility {id} with invalid travel_time_budget: {travel_time_budget}")
                continue

            # Create a new service area analysis layer
            layer_name = "Service_Area_Layer"
            arcpy.Delete_management(layer_name) if arcpy.Exists(layer_name) else None
            result_object = arcpy.na.MakeServiceAreaAnalysisLayer(network_dataset, layer_name, 'Drive',
                                                                "FROM_FACILITIES", [travel_time_budget]) # travel_mode
            service_area_layer = result_object.getOutput(0)

            # Add the current facility to the service area layer
            arcpy.management.MakeFeatureLayer(facilities, "current_facility_layer", f"hptripid = '{id}'")
            field_mappings = arcpy.na.NAClassFieldMappings(service_area_layer, "Facilities")
            field_mappings["Name"].mappedFieldName = "hptripid"
            arcpy.na.AddLocations(service_area_layer, "Facilities", "current_facility_layer", field_mappings)

            # Solve the service area
            arcpy.na.Solve(service_area_layer)

            # Check if the Polygons sublayer is available
            sublayers = {sublayer.name: sublayer for sublayer in service_area_layer.listLayers()}
            if "Polygons" in sublayers:
                polygons_sublayer = sublayers["Polygons"]
                arcpy.AddField_management(polygons_sublayer, "hptripid", "TEXT")
                arcpy.AddField_management(polygons_sublayer, "travel_time_budget", "DOUBLE")

                # Populate 'hptripid' and 'travel_time_budget' field in the Polygons sublayer
                with arcpy.da.UpdateCursor(polygons_sublayer, ["hptripid", "travel_time_budget"]) as update_cursor:
                    for update_row in update_cursor:
                        update_row[0], update_row[1] = id, travel_time_budget
                        update_cursor.updateRow(update_row)

                # Append the service area polygons to the ServiceArea feature class
                arcpy.management.Append(polygons_sublayer, service_area_fc, 'NO_TEST')
                arcpy.management.Delete(layer_name)
            else:
                print(f"No Polygons sublayer found for {mode} with {field_name} at {id}.")
                continue  # Skip to the next iteration

    # Save the updated feature class as a shapefile in the specified output directory
    output_file_path = os.path.join(output_dir, output_file_name)
    arcpy.CopyFeatures_management(service_area_fc, output_file_path)

    # Calculate processing time
    print(f"The script took {time.time() - start_time} seconds to complete for {mode} mode with {field_name}.")

# Call the Model function to execute the script for all modes
def Model():
    modes =['Auto']#['Bike', 'Walk', 'Transit', 'Auto']
    for mode in modes:
        orig_facilities = rf"E:\STP_PWoD\Data\Data_shapefile\Origin_shp\origin_{mode}_shp\origin_{mode}.shp"
        dest_facilities = rf"E:\STP_PWoD\Data\Data_shapefile\Destn_shp\destn_{mode}_shp\destn_{mode}.shp"

        generate_service_area(mode, 'Serv_Tbig', f'{mode}_Orig_serTbig.shp', facilities=orig_facilities)
        generate_service_area(mode, 'Serv_Tsm', f'{mode}_Orig_serTsm.shp', facilities=orig_facilities)
        generate_service_area(mode, 'Serv_Tbig', f'{mode}_Destn_SerTbig.shp', facilities=dest_facilities)
        generate_service_area(mode, 'Serv_Tsm', f'{mode}_Destn_SerTsm.shp', facilities=dest_facilities)

if __name__ == '__main__':
    Model()

ExecuteError: ERROR 030024: Solve returned a failure.
Insufficient number of valid locations in "Facilities".
Failed to execute (Solve).
